In [1]:
best_config = {
 'out_channel0': 16,
 'M': 4,
 'R1': 2,
 'R2': 0,
 'R3': 1,
 'R4': 2,
 'R5': 0,
 'convblock1': 1,
 'widenfact1': 0.5330917238012098,
 'B1': 2,
 'convblock2': 2,
 'widenfact2': 0.5289583249829364,
 'B2': 1,
 'convblock3': 2,
 'widenfact3': 0.6765230043009361,
 'B3': 3,
 'convblock4': 2,
 'widenfact4': 0.7745942817441474,
 'B4': 1,
 'convblock5': 0,
 'widenfact5': 0,
 'B5': 0
 }

In [2]:
from analogainas.search_spaces.resnet_macro_architecture import Network
from analogainas.utils import *

import torch
from torchsummary import summary
# Building the Optimal Sub-network

model = Network(best_config)
# model = model.cuda()
# summary(model, input_size=(3, 32, 32))

In [3]:
total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(total_params)

862838


In [4]:
from aihwkit.nn import AnalogConv2d, AnalogLinear, AnalogSequential
from aihwkit.nn.conversion import convert_to_analog_mapped, convert_to_analog
from aihwkit.optim import AnalogSGD
from torch.optim import SGD
from aihwkit.simulator.configs import FloatingPointRPUConfig, SingleRPUConfig, UnitCellRPUConfig, InferenceRPUConfig, DigitalRankUpdateRPUConfig
from aihwkit.simulator.configs.devices import *
from aihwkit.simulator.configs.utils import PulseType
# from aihwkit.simulator.rpu_base import cuda
from aihwkit.inference import BaseNoiseModel, PCMLikeNoiseModel, StateIndependentNoiseModel
from aihwkit.simulator.configs.utils import WeightClipType,WeightModifierType, IOParameters
from aihwkit.inference.compensation.drift import GlobalDriftCompensation

from aihwkit.simulator.configs.utils import BoundManagementType
from aihwkit.simulator.presets.utils import PresetIOParameters
import math

In [5]:
def create_rpu_config_new():
    rpu_config = InferenceRPUConfig()

    rpu_config.clip.type = WeightClipType.FIXED_VALUE
    rpu_config.clip.fixed_value = 1.0
    rpu_config.modifier.pdrop = 0  # Drop connect.

    rpu_config.modifier.std_dev = 0.5

    rpu_config.modifier.rel_to_actual_wmax = True
    rpu_config.mapping.digital_bias = True
    rpu_config.mapping.weight_scaling_omega = 0.4
    rpu_config.mapping.weight_scaling_omega = True
    rpu_config.mapping.max_input_size = 256
    rpu_config.mapping.max_output_size = 256

    rpu_config.mapping.learn_out_scaling_alpha = True

    rpu_config.forward = PresetIOParameters()
    rpu_config.forward.inp_res = 1/256  # 8-bit DAC discretization.
    rpu_config.forward.out_res = 1/256  # 8-bit ADC discretization.
    rpu_config.forward.bound_management = BoundManagementType.NONE

    # Inference noise model.
    rpu_config.noise_model = PCMLikeNoiseModel(g_max=25)

    # drift compensation
    rpu_config.drift_compensation = GlobalDriftCompensation()
    return rpu_config

def create_analog_optimizer(model, lr):
    """Create the analog-aware optimizer.

    Args:
        model (nn.Module): model to be trained

    Returns:
        Optimizer: created analog optimizer
    """

    optimizer = AnalogSGD(model.parameters(), lr) # we will use a learning rate of 0.01 as in the paper
    optimizer.regroup_param_groups(model)

    return optimizer

In [6]:
# model_analog

In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.backends.cudnn as cudnn
from torch.optim.lr_scheduler import CosineAnnealingLR
import torch.nn.functional as F
import pandas as pd
from analogainas.search_spaces.dataloaders.dataloader import load_nuclei_dataset
from tqdm import tqdm
from collections import OrderedDict

In [8]:
def train(train_loader, model, criterion, optimizer):
    avg_meters = {"loss": AverageMeter(), "iou": AverageMeter()}

    model.train()

    pbar = tqdm(total=len(train_loader))
    for input, target, _ in train_loader:
        input = input.to(device)
        target = target.to(device)

        output = model(input)

        loss = criterion(output, target)
        iou = iou_score(output, target)

        # compute gradient and do optimizing step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        avg_meters["loss"].update(loss.item(), input.size(0))
        avg_meters["iou"].update(iou, input.size(0))

        postfix = OrderedDict(
            [
                ("loss", avg_meters["loss"].avg),
                ("iou", avg_meters["iou"].avg),
            ]
        )
        pbar.set_postfix(postfix)
        pbar.update(1)
    pbar.close()

    return OrderedDict(
        [("loss", avg_meters["loss"].avg), ("iou", avg_meters["iou"].avg)]
    )


def test(val_loader, model, criterion):
    global best_acc
    avg_meters = {"loss": AverageMeter(), "iou": AverageMeter()}

    # switch to evaluate mode
    model.eval()

    with torch.no_grad():
        pbar = tqdm(total=len(val_loader))
        for input, target, _ in val_loader:
            input = input.to(device)
            target = target.to(device)

            output = model(input)
            loss = criterion(output, target)
            iou = iou_score(output, target)

            avg_meters["loss"].update(loss.item(), input.size(0))
            avg_meters["iou"].update(iou, input.size(0))

            postfix = OrderedDict(
                [
                    ("loss", avg_meters["loss"].avg),
                    ("iou", avg_meters["iou"].avg),
                ]
            )
            pbar.set_postfix(postfix)
            pbar.update(1)
        pbar.close()

    return OrderedDict(
        [("loss", avg_meters["loss"].avg), ("iou", avg_meters["iou"].avg)]
    )


In [9]:
def iou_score(output, target):
    smooth = 1e-5

    if torch.is_tensor(output):
        output = torch.sigmoid(output).data.cpu().numpy()
    if torch.is_tensor(target):
        target = target.data.cpu().numpy()
    output_ = output > 0.5
    target_ = target > 0.5
    intersection = (output_ & target_).sum()
    union = (output_ | target_).sum()

    return (intersection + smooth) / (union + smooth)

class BCEDiceLoss(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, input, target):
        bce = F.binary_cross_entropy_with_logits(input, target)
        smooth = 1e-5
        input = torch.sigmoid(input)
        num = target.size(0)
        input = input.view(num, -1)
        target = target.view(num, -1)
        intersection = input * target
        dice = (2.0 * intersection.sum(1) + smooth) / (
            input.sum(1) + target.sum(1) + smooth
        )
        dice = 1 - dice.sum() / num
        return 0.5 * bce + dice

class AverageMeter(object):
    """Computes and stores the average and current value"""

    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [10]:
import aihwkit
aihwkit.simulator.rpu_base.cuda.is_compiled()

False

In [11]:
# from aihwkit.simulator.tiles.transfer import TorchTransferTile
rpu_config = create_rpu_config_new()
# rpu_config.tile_class = TorchTransferTile
model_analog = convert_to_analog_mapped(model, rpu_config)
from aihwkit.utils.analog_info import analog_summary

analog_summary(model_analog,(1, 3, 32, 32))

Model Name: AnalogWrapperNetwork
Per-layer Information
Layer Information                                                     | Tile Information              
Layer Name          Is Analog           In Shape            Out Shape           Kernel Shape        # of Tiles          Reuse Factor        Log. tile shape     Phys. tile shape    utilization (%)     
AnalogConv2dMapped  analog              [1, 3, 32, 32]      [1, 16, 28, 28]     (7, 7)              1                   784                 -                   -                   -                   
                                                                                                                                            (16, 147)           (256, 256)          3.59                
BatchNorm2d         digital             [1, 16, 28, 28]     [1, 16, 28, 28]     -                   0                   0                   -                   -                   -                   
BatchNorm2d         digital           

In [15]:
# name = name + "_analog"
# rpu_config = create_rpu_config_new()
# model_analog = convert_to_analog_mapped(model, rpu_config)
# add this sequential to enable drift analog weights


# device = "cuda" if torch.cuda.is_available() else "cpu"
best_iou = 0
device = "cpu"
model_analog = AnalogSequential(model_analog)
trainloader, testloader = load_nuclei_dataset()
lr = 0.1
epochs = 20
model_analog.train()
model_analog.to(device)
params = filter(lambda p: p.requires_grad, model_analog.parameters())
optimizer = torch.optim.Adam(params, 1e-4)
optimizer = create_analog_optimizer(model_analog, lr = lr)

scheduler = CosineAnnealingLR(optimizer, T_max=400)
criterion = BCEDiceLoss()

log = OrderedDict([
    ('epoch', []),
    ('lr', []),
    ('loss', []),
    ('iou', []),
    ('val_loss', []),
    ('val_iou', []),
])

for epoch in range(0, epochs):
    train_log = train(trainloader, model_analog, criterion, optimizer)
    val_log = test(testloader, model_analog, criterion)
    model_analog.remap_analog_weights()
    scheduler.step()
    print('loss %.4f - iou %.4f - val_loss %.4f - val_iou %.4f'
          % (train_log['loss'], train_log['iou'], val_log['loss'], val_log['iou']))
    log['epoch'].append(epoch)
    log['loss'].append(train_log['loss'])
    log['iou'].append(train_log['iou'])
    log['val_loss'].append(val_log['loss'])
    log['val_iou'].append(val_log['iou'])
    if val_log['iou'] > best_iou:
        torch.save(model_analog.state_dict(), 'modelanalog.pth')
        best_iou = val_log['iou']
        print("=> saved best model")
        trigger = 0

/scratch/vg2507/HPML/analog-nas-unet


100%|██████████| 3/3 [00:57<00:00, 19.06s/it, loss=1.12, iou=2.04e-9]


loss 0.9295 - iou 0.2461 - val_loss 1.1158 - val_iou 0.0000
=> saved best model


100%|██████████| 3/3 [00:57<00:00, 19.23s/it, loss=1.08, iou=2.04e-9]


loss 0.9117 - iou 0.2523 - val_loss 1.0796 - val_iou 0.0000


...

loss 0.8121 - iou 0.3158 - val_loss 0.8897 - val_iou 0.2591


100%|██████████| 3/3 [00:57<00:00, 19.07s/it, loss=0.796, iou=0.348]


loss 0.7730 - iou 0.3517 - val_loss 0.7956 - val_iou 0.3477
=> saved best model


100%|██████████| 3/3 [00:57<00:00, 19.01s/it, loss=0.824, iou=0.349]

loss 0.7663 - iou 0.3575 - val_loss 0.8244 - val_iou 0.3489
=> saved best model


In [ ]:
from collections import OrderedDict
from tqdm import tqdm

def test_inference(config, model, criterion, test_loader):
    #model.eval()  # ensure the model is in evaluation mode

    # Initializing metric trackers
    avg_meters = {'loss': AverageMeter(),
                  'iou': AverageMeter(),
                  'accuracy': AverageMeter(),
                  'error': AverageMeter()}

    with torch.no_grad():
        pbar = tqdm(total=len(test_loader))

        # Simulation of inference at different times after training.
        for t_inference in [1, 3600,3600*24, 3600*24*30]:  # Example: Simulate the drift for 1 day.
            print(t_inference)
            model.drift_analog_weights(t_inference)  # Apply the drift simulation if applicable.

            for data_batch in test_loader:
                # Adjust the unpacking to account for the third item in the batch.
                images, labels, _ = data_batch  # The third item is ignored as in training.

                images = images.cuda()
                labels = labels.cuda()

                # Compute model output
                if config['deep_supervision']:
                    outputs = model(images)
                    loss = 0
                    for output in outputs:
                        loss += criterion(output, labels)
                    loss /= len(outputs)
                    iou = iou_score(outputs[-1], labels)
                else:
                    output = model(images)
                    loss = criterion(output, labels)
                    iou = iou_score(output, labels)

                # Calculate accuracy and error
                _, predicted = torch.max(output.data, 1)
                total = labels.size(0)
                correct = (predicted == labels).sum().item()
                accuracy = correct / total
                error = 1 - accuracy

                # Update tracking variables
                avg_meters['loss'].update(loss.item(), total)
                avg_meters['iou'].update(iou, total)

                pbar.update(1)

            # Displaying statistics after inference
            print(f'Inference Time: {t_inference: .2e} seconds')
            print(f'Average Loss: {avg_meters["loss"].avg:.4f}\tAverage IoU: {avg_meters["iou"].avg:.4f}')

            pbar.close()

            # Resetting the average meters for the next inference time point
            for meter in avg_meters.values():
                meter.reset()

    return OrderedDict([('loss', avg_meters['loss'].avg),
                        ('iou', avg_meters['iou'].avg)])

test_inference(best_config, model_analog,criterion, val_loader )